In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import modules.logger_tool as logger
import os

os.environ['LOG_NAME'] = 'science'
os.environ['LOG_DIR'] = 'logs'
os.environ['LOG_LEVEL'] = 'DEBUG'

logging = logger.get_logger(os.environ['LOG_NAME'], log_level=os.environ['LOG_LEVEL'], log_path=os.environ['LOG_DIR'], log_file=os.environ['LOG_NAME'])


In [ ]:
import modules.database.schemas.curriculum_neo as neo_curriculum
import modules.database.schemas.relationships.curricular_relationships as neo_relationships
import modules.database.tools.xl_planner_tools as planner
import modules.database.tools.neontology_tools as neon
import modules.database.tools.neo4j_driver_tools as driver_tools
import modules.database.tools.neo4j_session_tools as session_tools

import requests
from pydantic import ValidationError


In [ ]:
db_name = 'science'

In [ ]:
url = 'http://localhost:9500/database/admin/stop-database'
data = {'db_name': db_name}
response = requests.post(url, json=data)
logging.info(response.text)

url = 'http://localhost:9500/database/admin/drop-database'
data = {'db_name': db_name}
response = requests.post(url, json=data)
logging.info(response.text)

In [ ]:
url = 'http://localhost:9500/database/admin/create-database'
params = {'db_name': db_name}
response = requests.post(url, params=params)
logging.info(response.text)

In [ ]:
driver = driver_tools.get_driver(database=db_name)


In [ ]:
neon.init_neo4j_connection()


In [ ]:
# Create KeyStageNodes from a list
key_stages = [
    neo_curriculum.KeyStageNode(key_stage_level=3, key_stage='KS3'),
    neo_curriculum.KeyStageNode(key_stage_level=4, key_stage='KS4'),
    neo_curriculum.KeyStageNode(key_stage_level=5, key_stage='KS5')
]
# Now create a routine to create key stages from a list
for key_stage in key_stages:
    neon.create_or_merge_neontology_node(key_stage, database=db_name, operation='create')


In [ ]:
# Create SubjectNodes from a list
subjects = [
    neo_curriculum.SubjectNode(subject_id='SCI', subject_name='Science'),
    neo_curriculum.SubjectNode(subject_id='PHY', subject_name='Physics'),
    neo_curriculum.SubjectNode(subject_id='CHE', subject_name='Chemistry'),
    neo_curriculum.SubjectNode(subject_id='BIO', subject_name='Biology'),
    neo_curriculum.SubjectNode(subject_id='MAT', subject_name='Mathematics'),
    neo_curriculum.SubjectNode(subject_id='ENG', subject_name='English'),
    neo_curriculum.SubjectNode(subject_id='HIS', subject_name='History')
]
# Now create a routine to create subjects from a list
for subject in subjects:
    neon.create_or_merge_neontology_node(subject, database=db_name, operation='create')


In [ ]:
excel_planner = planner.create_dataframes(os.getenv("EXCEL_PLANNER_FILE"))

topic_df = excel_planner['topiclookup_df']
lesson_df = excel_planner['lessonlookup_df']
statement_df = excel_planner['statementlookup_df']


In [ ]:
topic_excel_node_list = []
topic_excel_helper_list = []
topic_lesson_excel_node_list = []
topic_lesson_excel_helper_list = []
learning_statement_excel_node_list = []
learning_statement_excel_helper_list = []

default_topic_values = {
    'topic_assessment_type': 'Null'
    }

default_topic_lesson_values = {
    'topic_lesson_title': 'Null',  # Corrected default value key
    'topic_lesson_type': 'Null',  # Corrected default value key
    'topic_lesson_length': 1,             # Corrected default value key
    'topic_lesson_suggested_activities': 'Null',  # Corrected default value key
    'topic_lesson_skills_learned': 'Null',  # Corrected default value key
    'topic_lesson_weblinks': 'Null',   # Corrected default value key
}

default_learning_statement_values = {
    # Add default values for fields that might contain NaN
    'lesson_learning_statement': 'Null',
    'lesson_learning_statement_type': 'Student learning outcome'
}


In [ ]:
# Process each topic and related lessons and statements
for index, topic_row in topic_df.iterrows():
    # Create TopicNode
    topic_node_data = {
        'topic_id': topic_row.get('TopicID'),
        'topic_title': topic_row.get('TopicTitle'),
        'total_number_of_lessons_for_topic': topic_row.get('TotalNumberOfLessonsForTopic'),
        'topic_type': topic_row.get('TopicType'),
        'topic_assessment_type': topic_row.get('TopicAssessmentType')
    }
    topic_node_data = planner.replace_nan_with_default(topic_node_data, default_topic_values)
    topic_node = neo_curriculum.TopicNode(**topic_node_data)

    try:
        # Create or merge TopicNode in the database
        neon.create_or_merge_neontology_node(topic_node, database=db_name, operation='merge')

        # Process lessons for this topic
        for lesson_index, lesson_row in lesson_df[lesson_df['TopicID'] == topic_row['TopicID']].iterrows():
            lesson_node_data = {
                'topic_lesson_id': lesson_row.get('LessonID'),
                'topic_lesson_title': lesson_row.get('LessonTitle', default_topic_lesson_values['topic_lesson_title']),
                'topic_lesson_type': lesson_row.get('TopicLessonType', default_topic_lesson_values['topic_lesson_type']),
                'topic_lesson_length': lesson_row.get('SuggestedNumberOfPeriodsForLesson', default_topic_lesson_values['topic_lesson_length']),
                'topic_lesson_suggested_activities': lesson_row.get('SuggestedActivities', default_topic_lesson_values['topic_lesson_suggested_activities']),
                'topic_lesson_skills_learned': lesson_row.get('SkillsLearned', default_topic_lesson_values['topic_lesson_skills_learned']),
                'topic_lesson_weblinks': lesson_row.get('TopicLessonWeblinks', default_topic_lesson_values['topic_lesson_weblinks'])
            }
            lesson_node_data = planner.replace_nan_with_default(lesson_node_data, default_topic_lesson_values)
            lesson_node = neo_curriculum.TopicLessonNode(**lesson_node_data)

            try:
                # Create TopicLessonNode in the database
                neon.create_or_merge_neontology_node(lesson_node, database=db_name, operation='merge')

                # Process statements for this lesson
                for statement_index, statement_row in statement_df[statement_df['LessonID'] == lesson_row['LessonID']].iterrows():
                    statement_node_data = {
                        'lesson_learning_statement_id': statement_row.get('StatementID'),
                        'lesson_learning_statement': statement_row.get('LearningOutcomeStatement', default_learning_statement_values['lesson_learning_statement']),
                        'lesson_learning_statement_type': statement_row.get('LearningStatementType', default_learning_statement_values['lesson_learning_statement_type'])
                    }
                    statement_node_data = planner.replace_nan_with_default(statement_node_data, default_learning_statement_values)
                    statement_node = neo_curriculum.LearningStatementNode(**statement_node_data)

                    try:
                        # Create LearningStatementNode in the database
                        neon.create_or_merge_neontology_node(statement_node, database=db_name, operation='merge')

                        # Create relationships
                        # Relationship between TopicNode and TopicLessonNode
                        topic_has_lesson_relationship = neo_relationships.TopicIncludesTopicLesson(source=topic_node, target=lesson_node)
                        neon.create_or_merge_neontology_relationship(topic_has_lesson_relationship, database=db_name, operation='merge')

                        # Relationship between TopicLessonNode and LearningStatementNode
                        statement_in_lesson_relationship = neo_relationships.LearningStatementPartOfTopicLesson(source=statement_node, target=lesson_node)
                        neon.create_or_merge_neontology_relationship(statement_in_lesson_relationship, database=db_name, operation='merge')

                    except ValidationError as e:
                        logging.error(f"Validation error for statement row {statement_index}: {e}")

            except ValidationError as e:
                logging.error(f"Validation error for lesson row {lesson_index}: {e}")

    except ValidationError as e:
        logging.error(f"Validation error for topic row {index}: {e}")